<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from gensim.models import LsiModel
from gensim.models import Word2Vec

In [2]:
max_seq_length = 100

In [3]:
ratings1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/ratings.csv')
ratings = ratings1.iloc[0:1000,:]
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


**Sorting the dataframe**

In [4]:
ratings.sort_values(by=['userId','timestamp'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
36,1,5952,4.0,1147868053
13,1,2012,2.5,1147868068
12,1,2011,2.5,1147868079
11,1,1653,4.0,1147868097
9,1,1250,4.0,1147868414
40,1,6539,3.5,1147868461
39,1,6377,4.0,1147868469
21,1,3448,4.0,1147868480
5,1,1088,4.0,1147868495
4,1,899,3.5,1147868510


In [6]:
users_total = len(ratings['userId'].unique())
print(users_total)

4


**Selecting the most recent movies rated by each user and padding if necessary**

In [7]:
movie_list = []
for i in range(users_total):
  list1 = []
  list1 = ratings.loc[ratings['userId'] ==(i+1),['movieId']]['movieId'].tolist()
  if len(list1)>max_seq_length:
    list1 = list1[(len(list1)-max_seq_length):]
  elif len(list1)<max_seq_length:
    for j in range((max_seq_length-len(list1))):
      list1.append(0)
  movie_list.append(list1)
#print(movie_list)

**Selecting the most recent ratings of each user and padding if necessary**

In [8]:
rating_list =[]
for i in range(users_total):
  list2 = []
  list2 = ratings.loc[ratings['userId'] ==(i+1),['rating']]['rating'].tolist()
  if len(list2)>max_seq_length:
    list2 = list2[(len(list2)-max_seq_length):]
  elif len(list2)<max_seq_length:
    for j in range((max_seq_length-len(list2))):
      list2.append(0)
  rating_list.append(list2)
#print(rating_list)

In [9]:
movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(users_total)])
movies_transpose.index.names = ['userId']
print(movies_transpose)

            0     1     2       3     4   ...     95      96     97    98    99
userId                                    ...                                  
1         5952  2012  2011    1653  1250  ...      0       0      0     0     0
2         4306  8368  8360    3793  4995  ...   2987    1293   7090  1136  2150
3       176371  3968  8983  102407  6548  ...  88125  106489  81834  4262  1732
4         8665  8961  4993    1580  5444  ...      0       0      0     0     0

[4 rows x 100 columns]


In [10]:
ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(users_total)])
ratings_transpose.index.names = ['userId']
print(ratings_transpose)

         0    1    2    3    4    5    6   ...   93   94   95   96   97   98   99
userId                                     ...                                   
1       4.0  2.5  2.5  4.0  4.0  3.5  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       4.5  5.0  4.0  5.0  5.0  4.0  5.0  ...  0.5  3.0  0.5  5.0  2.0  5.0  5.0
3       4.5  3.0  4.0  4.0  3.0  4.0  3.5  ...  2.5  4.0  3.5  3.5  3.5  4.5  4.5
4       3.5  5.0  4.5  4.5  4.0  4.0  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4 rows x 100 columns]


In [11]:
# Select features from original dataset to form a new dataframe 
df1 = ratings_transpose.iloc[:]# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [12]:
#list of lists format of the ratings transpose matrix
print(sent)

[['4.0', '2.5', '2.5', '4.0', '4.0', '3.5', '4.0', '4.0', '4.0', '3.5', '3.0', '3.5', '5.0', '5.0', '4.0', '4.0', '3.5', '3.5', '5.0', '5.0', '3.5', '5.0', '4.5', '4.5', '5.0', '4.0', '2.5', '3.0', '4.5', '5.0', '3.0', '5.0', '3.0', '3.5', '3.5', '3.5', '2.5', '4.5', '4.0', '4.0', '3.5', '4.0', '4.0', '3.5', '4.5', '4.5', '4.0', '1.0', '2.5', '3.0', '2.5', '3.5', '5.0', '5.0', '3.5', '3.5', '3.0', '5.0', '5.0', '4.0', '4.5', '5.0', '5.0', '0.5', '5.0', '3.0', '2.0', '2.0', '5.0', '5.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0'], ['4.5', '5.0', '4.0', '5.0', '5.0', '4.0', '5.0', '4.5', '4.0', '5.0', '4.0', '4.5', '5.0', '4.0', '4.0', '5.0', '3.0', '4.0', '3.5', '4.0', '4.5', '5.0', '4.5', '3.5', '5.0', '4.5', '4.5', '5.0', '4.0', '3.0', '3.0', '4.0', '3.0', '4.5', '3.0', '4.0', '4.5', '4.5', '2.0', '5.0', '3.5', '5.0', '0

In [13]:
model = Word2Vec(sent,min_count=1,size= 50,workers=3, window =3, sg = 1)

In [14]:
model.save('wordtovec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
lsi = models.LsiModel(corpus=None, num_topics=10, id2word=None) # LSI Model Training 
index = similarities.MatrixSimilarity(lsi[corpus])
train_vectors = lsi[corpus]

NameError: ignored